In [5]:
import pandas as pd
import DataLoading
import NN_evaluation

In [2]:
sc = "sc01"
period = "2030"
case= "RTS24"
folder = f"../Data/{case}_DC_fy/"

all_executions = DataLoading.list_executions(folder=folder,per = period,sc=sc)
te_s = 0.3
val_s = 0.3
dfs_in, dfs_out, dfs_inter = DataLoading.load_data_ext_out(folder, all_executions, period, sc, ["PowerFlow"],"SystemCosts")
dfs_inter_j = DataLoading.join_frames_inter_layer(dfs_inter,all_executions)
dfs_inter_j = DataLoading.trim_columns_to_common(dfs_inter_j)
# Convert to pytorch tensors
ts_in, ts_out, ts_inter = DataLoading.split_tr_val_te_ext_out(dfs_in, dfs_out, dfs_inter_j, all_executions, te_s, val_s)
d_ft_in, d_ft_out, d_ft_inter,maxs = DataLoading.concat_and_normalize_ext_out(ts_in, ts_out, ts_inter, all_executions)


input_f_sc01_Network_Existing_Generation_Full_2030.csv
90
input_f_sc01_Network_Full_Generation_Full_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_101_N_102_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_101_N_103_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_101_N_105_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_102_N_104_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_102_N_106_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_103_N_109_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_103_N_124_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_104_N_109_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_105_N_110_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_106_N_108_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_106_N_110_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_107_N_108_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_108_N_109_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_108_N_110_cac1_2030.csv
90
input_

In [6]:
existing_actual = NN_evaluation.get_actual_from_dfs_out(all_executions[0],dfs_out)/maxs["out_scalar"]
full_actual = NN_evaluation.get_actual_from_dfs_out(all_executions[1],dfs_out)/maxs["out_scalar"]

In [19]:
ics = DataLoading.list_investments_candidates_from_execs(executions=all_executions)
df_lb = pd.DataFrame()
for ic in ics:
    ix_P = [i for i, string in enumerate(all_executions) if ic in string and "PINT" in string]
    ix_T = [i for i, string in enumerate(all_executions) if ic in string and "TOOT" in string]
    ex_P  = all_executions[ix_P[0]]
    ex_T = all_executions[ix_T[0]]

    ex_actual_P = NN_evaluation.get_actual_from_dfs_out(ex_P,dfs_out)/maxs["out_scalar"]
    ex_actual_T = NN_evaluation.get_actual_from_dfs_out(ex_T,dfs_out)/maxs["out_scalar"]

    lb_P = existing_actual -ex_actual_P
    lb_T = - (full_actual - ex_actual_T)

    print(ic, "PINT: ", lb_P.sum().item(),"TOOT:",lb_T.sum().item())

    lb = (lb_P + lb_T)/2 * maxs["out_scalar"]
    df_lb[ic] = lb

N_101_N_102_cac1 PINT:  -0.010670835701521279 TOOT: -0.0019388914257595165
N_101_N_103_cac1 PINT:  -0.5152647559386263 TOOT: -0.10757143669345778
N_101_N_105_cac1 PINT:  -0.061968595592738415 TOOT: 0.00046399417887322506
N_102_N_104_cac1 PINT:  -0.0006119093589996749 TOOT: 0.002955782748516878
N_102_N_106_cac1 PINT:  -0.0908710764399161 TOOT: 0.002637501557014427
N_103_N_109_cac1 PINT:  -0.410442730196161 TOOT: -0.11080886355734364
N_103_N_124_cac1 PINT:  -6.009733228582405 TOOT: -0.5344665439459617
N_104_N_109_cac1 PINT:  0.04139970512276473 TOOT: 0.008422995376953988
N_105_N_110_cac1 PINT:  0.07952303380387442 TOOT: 0.019009868129412914
N_106_N_108_cac1 PINT:  -0.025758427604322155 TOOT: -0.001660742504840179
N_106_N_110_cac1 PINT:  0.08869367787220685 TOOT: 0.010492463013138484
N_107_N_108_cac1 PINT:  10.90267291173012 TOOT: 2.9627666721076267
N_108_N_109_cac1 PINT:  0.03917889823816305 TOOT: -0.0033545158417240136
N_108_N_110_cac1 PINT:  -0.07234789001189523 TOOT: -0.00082707171293

In [20]:
df_lb.sum()

N_101_N_102_cac1       -5.540784
N_101_N_103_cac1     -273.677696
N_101_N_105_cac1      -27.025465
N_102_N_104_cac1        1.029911
N_102_N_106_cac1      -38.770325
N_103_N_109_cac1     -229.040856
N_103_N_124_cac1    -2875.557866
N_104_N_109_cac1       21.892372
N_105_N_110_cac1       43.295906
N_106_N_108_cac1      -12.048136
N_106_N_110_cac1       43.582943
N_107_N_108_cac1     6092.551457
N_108_N_109_cac1       15.741433
N_108_N_110_cac1      -32.153486
N_109_N_111_cac1      207.459242
N_109_N_112_cac1      151.369920
N_110_N_111_cac1      176.824015
N_110_N_112_cac1      104.763340
N_111_N_113_cac1        1.384676
N_111_N_114_cac1      335.559350
N_112_N_113_cac1       14.188695
N_112_N_123_cac1      356.941713
N_113_N_123_cac1      256.746649
N_114_N_116_cac1      558.163656
N_115_N_116_cac1    -4360.529877
N_115_N_121_cac1    24449.280901
N_115_N_121_cac2    24449.280901
N_115_N_124_cac1      855.902453
N_116_N_117_cac1    12630.379624
N_116_N_119_cac1      180.297986
N_117_N_11

In [ ]:
_path_0 

In [ ]:
table.to_csv(_path_0+'/3.Out'+'/'+'oT_LineBenefit_Data_'+CaseName_Base+'.csv')